In [ ]:
import pandas as pd
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [ ]:
df_train = df_train.dropna()

In [ ]:
df_train.drop(['case_id', 'patientid', 'Ward_Facility_Code', 'Hospital_type_code', 'City_Code_Hospital' ], axis=1, inplace=True)
df_test.drop(['case_id', 'patientid' , 'Ward_Facility_Code', 'Hospital_type_code', 'City_Code_Hospital' ], axis=1, inplace=True)

In [ ]:
#df_train['Severity of Illness'].unique() 
m2 = {'Emergency':0, 'Urgent':1, 'Trauma':2}
m3 = {'Minor':0, 'Moderate':1, 'Extreme':2}

df_train['Type of Admission'] = df_train['Type of Admission'].map(m2)
df_test['Type of Admission'] = df_test['Type of Admission'].map(m2)

df_train['Severity of Illness'] = df_train['Severity of Illness'].map(m3)
df_test['Severity of Illness'] = df_test['Severity of Illness'].map(m3)

In [ ]:
m = {'0-10':0, '11-20':1, '21-30':2, '31-40':3,'41-50':4,'51-60':5,'61-70':6,'71-80':7,'81-90':8,'91-100':9,'More than 100 Days':10 }
df_train['Age'] = df_train['Age'].map(m)
df_train['Stay'] = df_train['Stay'].map(m)

df_test['Age'] = df_test['Age'].map(m)

In [ ]:
from sklearn.preprocessing import LabelEncoder

df_train["Hospital_region_code"] = le.fit_transform(df_train["Hospital_region_code"].values)
df_test["Hospital_region_code"] = le.transform(df_test["Hospital_region_code"].values)

df_train["Department"] = le.fit_transform(df_train["Department"].values)
df_test["Department"] = le.transform(df_test["Department"].values)

df_train["Ward_Type"] = le.fit_transform(df_train["Ward_Type"].values)
df_test["Ward_Type"] = le.transform(df_test["Ward_Type"].values)
'''
df_train["Ward_Facility_Code"] = le.fit_transform(df_train["Ward_Facility_Code"].values)
df_test["Ward_Facility_Code"] = le.transform(df_test["Ward_Facility_Code"].values)
'''
df_train["Type of Admission"] = le.fit_transform(df_train["Type of Admission"].values)
df_test["Type of Admission"] = le.transform(df_test["Type of Admission"].values)

df_train["Severity of Illness"] = le.fit_transform(df_train["Severity of Illness"].values)
df_test["Severity of Illness"] = le.transform(df_test["Severity of Illness"].values)

df_train["Age"] = le.fit_transform(df_train["Age"].values)
df_test["Age"] = le.transform(df_test["Age"].values)

cols_cat = [index_ for index_, string in enumerate(df_train.columns) if string in ["Hospital_type_code", "Hospital_region_code", "Department", "Ward_Type", "Ward_Facility_Code", "Type of Admission", "Severity of Illness", "Age"]]

In [ ]:
from sklearn.preprocessing import StandardScaler

s = StandardScaler()

df_train[['Hospital_code', 'Available Extra Rooms in Hospital','Bed Grade','City_Code_Patient', 'Visitors with Patient', 'Age', 'Admission_Deposit']] = s.fit_transform(df_train[['Hospital_code','Available Extra Rooms in Hospital','Bed Grade','City_Code_Patient', 'Visitors with Patient', 'Age', 'Admission_Deposit']].values)
df_test[['Hospital_code', 'Available Extra Rooms in Hospital','Bed Grade','City_Code_Patient', 'Visitors with Patient', 'Age', 'Admission_Deposit']] = s.transform(df_test[['Hospital_code', 'Available Extra Rooms in Hospital','Bed Grade','City_Code_Patient', 'Visitors with Patient', 'Age', 'Admission_Deposit']].values)

In [ ]:
df_labels = pd.get_dummies(df_train['Stay'].values)

In [ ]:
df_labels.head()

,0,1,2,3,4,5,6,7,8,9,10
0,1,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0


In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
 
X_train, Y = df_train.drop(["Stay"], axis=1).values, df_train["Stay"].values
X_test = df_test.values
Y = df_labels.values

In [ ]:
import keras
from keras.models import Model
from keras.layers import Input,  Dense , concatenate, Reshape, Flatten, BatchNormalization
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import Bidirectional
import numpy as np
import tensorflow

tensorflow.random.set_seed(2)
session_conf = tensorflow.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tensorflow.compat.v1.Session(graph=tensorflow.compat.v1.get_default_graph(), config=session_conf)
tensorflow.compat.v1.keras.backend.set_session(sess)

batch_size = 32

#categorical inputs
inp2 = Input(shape=(1,))
inp3 = Input(shape=(1,))
inp4 = Input(shape=(1,))
inp6 = Input(shape=(1,))
inp7 = Input(shape=(1,))

# numerical input
inp8 = Input(shape=(7,)) 

#embedding layers
layer2 = Flatten()(Embedding(len(df_train.Hospital_region_code.unique()), 2*len(df_train.Hospital_region_code.unique()), trainable=True)(inp2))
layer3 = Flatten()(Embedding(len(df_train.Department.unique()),2*len(df_train.Department.unique()), trainable=True)(inp3))
layer4 = Flatten()(Embedding(len(df_train.Ward_Type.unique()), 2*len(df_train.Ward_Type.unique()), trainable=True)(inp4))
layer6 = Flatten()(Embedding(len(df_train["Type of Admission"].unique()),2*len(df_train["Type of Admission"].unique()), trainable=True)(inp6))
layer7 = Flatten()(Embedding(len(df_train["Severity of Illness"].unique()), 2*len(df_train["Severity of Illness"].unique()), trainable=True)(inp7))

merge = concatenate([layer2,layer3,layer4,layer6,layer7,inp8])

opt = keras.optimizers.Adam(learning_rate=0.0005)
hidden1 = Dense(512, activation='relu')(merge)
hidden_test = Dense(256, activation='relu')(hidden1)
hidden2 = Dense(256, activation='relu')(hidden1)
hidden3 = Dense(128, activation='relu')(hidden2)
hidden4 = Dense(64, activation='relu')(hidden3)
hidden5 = Dense(32, activation='relu')(hidden4)


#concat both models
output = Dense(11,activation='softmax')(hidden5)
model = Model(inputs=[inp2,inp3,inp4,inp6,inp7,inp8], outputs=output)
#model = Model(inputs=[inp1,inp2,inp3,inp4,inp5,inp6,inp7,inp8], outputs=output)
model.compile(loss = 'categorical_crossentropy', optimizer=opt,metrics = ['accuracy'])
print(model.summary())

model.fit(x = [np.reshape(df_train.Hospital_region_code.values, (-1,1)),
               np.reshape(df_train.Department.values, (-1,1)),
               np.reshape(df_train.Ward_Type.values, (-1,1)),
               np.reshape(df_train["Type of Admission"].values, (-1,1)),
               np.reshape(df_train["Severity of Illness"].values, (-1,1)),
               df_train[['Hospital_code', 'Available Extra Rooms in Hospital','Bed Grade','City_Code_Patient', 'Visitors with Patient', 'Age', 'Admission_Deposit']]], 
          y = Y, batch_size =batch_size, epochs = 7,  verbose = 1)


In [ ]:
preds = model.predict([np.reshape(df_test.Hospital_region_code.values, (-1,1)),
               np.reshape(df_test.Department.values, (-1,1)),
               np.reshape(df_test.Ward_Type.values, (-1,1)),
               np.reshape(df_test["Type of Admission"].values, (-1,1)),
               np.reshape(df_test["Severity of Illness"].values, (-1,1)),
               df_test[['Hospital_code', 'Available Extra Rooms in Hospital','Bed Grade','City_Code_Patient', 'Visitors with Patient', 'Age', 'Admission_Deposit']]], verbose=1)

4284/4284 [==============================] - 8s 2ms/step


In [ ]:
preds = np.argmax(preds, axis = 1)
print(preds)

[0 5 2 ... 2 1 5]


In [ ]:
m_rev = {'0':'0-10', '1':'11-20', '2':'21-30', '3':'31-40','4':'41-50','5':'51-60','6':'61-70','7':'71-80','8':'81-90','9':'91-100','10':'More than 100 Days' }
#preds_final = m_rev[preds]

new_preds= []
for i in range(len(preds)):
  #new_preds.append(m_rev[str(preds[i][0])])
  new_preds.append(m_rev[str(preds[i])])


In [ ]:
df_submit = pd.DataFrame({'Stay': new_preds})
df_submit.to_excel("submit.xlsx", index=False)